# EDA and Visualization Final Project: Data Prep and Analysis
## Zachary Barnes and Bing Wang

##### Housekeeping

In [1]:
# Load Python libraries
import os
import numpy as np
import pandas as pd

# Run R code adjacent to Python code
%load_ext rpy2.ipython

# Load ggplot R library
%R library(ggplot2)
%R library(scales)

# Avoid kernal death
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Read in and organize data (Collisions and Victims datasets, from TIMS)

In [2]:
c = pd.read_csv("Collisions.csv")

# Keep only relevant columns
c = c[['CASE_ID','COLLISION_DATE','COLLISION_TIME','INTERSECTION','COLLISION_SEVERITY',
        'LIGHTING','LOCATION_TYPE','MVIW','TYPE_OF_COLLISION',
        'WEATHER_1','LATITUDE','LONGITUDE','PRIMARY_RD',
        'SECONDARY_RD','POINT_X','POINT_Y']]
# Convert COLLISION_TIME to hour of day (use tlater to match speeds to collisions)
c['COLLISION_TIME'] = [int(i[:-2]) if len(i) > 2 else 0 for i in c['COLLISION_TIME'].astype(str).values]

# Add quarter column (use later in match speeds to collisions)
c['QUARTER'] = c['COLLISION_DATE'].map(pd.to_datetime).dt.quarter
c.sample(5)

,CASE_ID,COLLISION_DATE,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,LIGHTING,LOCATION_TYPE,MVIW,TYPE_OF_COLLISION,WEATHER_1,LATITUDE,LONGITUDE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,QUARTER
1713,8685835,2018-06-19,14,N,4,A,NaN,B,D,A,NaN,NaN,15TH ST,VALENCIA ST,-122.422058,37.766528,2
1313,8716411,2018-09-07,23,Y,4,C,NaN,C,D,A,NaN,NaN,GEARY BL,2ND AV,-122.460020,37.781280,3
1657,8685903,2018-07-17,10,N,4,A,NaN,I,E,A,NaN,NaN,NORIEGA ST,30TH AV,-122.488495,37.753709,3
788,8756712,2018-11-25,2,Y,4,C,NaN,C,D,A,NaN,NaN,6TH ST,HOWARD ST,-122.407140,37.779750,4
2927,8575277,2018-02-21,14,Y,4,A,NaN,C,B,A,NaN,NaN,16TH ST,FLORIDA ST,-122.411440,37.765550,1


In [3]:
v = pd.read_csv('Victims12.csv')

# Keep only relevant columns
v = v[['CASE_ID','VICTIM_DEGREE_OF_INJURY','VICTIM_ROLE','VICTIM_AGE']]
v.sample(5)

,CASE_ID,VICTIM_DEGREE_OF_INJURY,VICTIM_ROLE,VICTIM_AGE
41,8758213,2,1,36
222,8594870,2,3,49
250,8572676,2,3,81
126,8667490,2,3,54
164,8636921,2,4,48


In [4]:
# Merge Collisions and Victims datasets
cv = pd.merge(c, v, how='left', on='CASE_ID')
cv.sample(5)

,CASE_ID,COLLISION_DATE,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,LIGHTING,LOCATION_TYPE,MVIW,TYPE_OF_COLLISION,WEATHER_1,LATITUDE,LONGITUDE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,QUARTER,VICTIM_DEGREE_OF_INJURY,VICTIM_ROLE,VICTIM_AGE
1942,8661472,2018-05-14,13,N,4,A,NaN,C,C,A,NaN,NaN,VAN DYKE AV,KEITH ST,-122.390916,37.72829,2,NaN,NaN,NaN
915,8754805,2018-10-18,13,Y,3,A,NaN,G,D,A,NaN,NaN,WASHBURN ST,MISSION ST,-122.415140,37.77589,4,NaN,NaN,NaN
2025,8656282,2018-06-13,19,Y,4,A,NaN,G,D,A,NaN,NaN,EL CAMINO DEL MAR,28TH AV,-122.488420,37.78763,2,NaN,NaN,NaN
1853,8667511,2018-06-07,23,Y,2,C,NaN,B,G,A,NaN,NaN,DIVISION ST,HAYES ST,NaN,NaN,2,2.0,3.0,31.0
406,8777274,2018-12-14,7,Y,4,A,NaN,C,C,A,NaN,NaN,HUDSON AV,MENDELL ST,-122.386400,37.73897,4,NaN,NaN,NaN


In [5]:
# Paul's Function to remove NaNs from columns
def fixDFcolsForR(myDF):
    for i in range(myDF.shape[1]): # for each col
        if myDF.iloc[:, i].dtype == 'O': # check if col is type object
            myDF.iloc[:, i] = myDF.iloc[:, i].astype(str) # if yes set type to string
    return myDF

In [6]:
# Convert NaNs to string type
cv = fixDFcolsForR(cv)
cv.sample(5)

,CASE_ID,COLLISION_DATE,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,LIGHTING,LOCATION_TYPE,MVIW,TYPE_OF_COLLISION,WEATHER_1,LATITUDE,LONGITUDE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,QUARTER,VICTIM_DEGREE_OF_INJURY,VICTIM_ROLE,VICTIM_AGE
1753,8683806,2018-07-07,23,N,3,C,nan,G,D,A,NaN,NaN,MISSION ST,8TH ST,-122.41316,37.77746,3,NaN,NaN,NaN
919,8754801,2018-10-18,18,Y,4,A,nan,B,G,A,NaN,NaN,FRANKLIN ST,PAGE ST,-122.42077,37.77429,4,NaN,NaN,NaN
2253,8636938,2018-05-21,16,Y,4,A,nan,C,C,A,NaN,NaN,JUNIPERO SERRA BL,PALMETTO AV,-122.47106,37.71064,2,NaN,NaN,NaN
3456,90737867,2018-05-18,22,N,4,D,nan,C,C,A,38.80843,-122.36704,I-80 W/B FROM TREASURE ISLAND RD,FOREST RD,NaN,NaN,2,NaN,NaN,NaN
3806,90877252,2018-10-18,10,N,3,A,R,C,B,A,37.72156,-122.44843,I-280 S/B TO GENEVA AVE.,GENEVA AVE,-122.44843,37.72156,4,NaN,NaN,NaN


Now, we need to attach Open Street Maps (OSM) way IDs to each observation in the Collisions-Victims dataset, in order to match with the speed dataset. An OSM way ID is an ID that matches latitude and longitude to a street segment.

# Use Nominatim API to convert lat long to OSM ID

In [9]:
import requests
import json

def osmid(lat,lon):
    URL = 'https://nominatim.openstreetmap.org/reverse?format=json&lat=%s&lon=%s'
    r = requests.get(URL % (lat,lon))
    data = json.loads(r.text)
#     osm_id = data.get('osm_id','NONE')
    return data['osm_id']


This code snippet below creates osm.csv, a list of OSM IDS and their latitudes and longitudes. We will use this to attach OSM IDs to our Collisions-Victims dataset.

In [ ]:
# import time
# lat_long = cv[['POINT_Y','POINT_X','VICTIM_DEGREE_OF_INJURY']]
# osm_id = list()
# for i,row in lat_long.iterrows():
#     if (np.isnan(row['POINT_Y']) or np.isnan(row['POINT_X'])) or np.isnan(row['VICTIM_DEGREE_OF_INJURY']):
#         osm_id.append('NONE')
#         continue
#     else:
#         time.sleep(2)
#         osm_id.append(osmid(row['POINT_Y'],row['POINT_X']))
# osm_series = pd.Series(osm_id)
# osm_series.to_csv('osm.csv')

# Add OSM_ID column 

In [20]:
osm = pd.read_csv('osm.csv',header=None)
osm.columns = ['index','OSM_ID']
cv['OSM_ID'] = osm['OSM_ID']
cv.sample(5)

,CASE_ID,COLLISION_DATE,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,LIGHTING,LOCATION_TYPE,MVIW,TYPE_OF_COLLISION,WEATHER_1,...,LONGITUDE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,QUARTER,VICTIM_DEGREE_OF_INJURY,VICTIM_ROLE,VICTIM_AGE,OSM_ID
2901,8594562,2018-03-12,15,Y,2,A,nan,C,D,B,...,NaN,LEAVENWORTH ST,BEACH ST,-122.41892,37.80687,1,2.0,1.0,62.0,103339440
1181,8716577,2018-09-29,23,Y,4,C,nan,E,D,A,...,NaN,OAKDALE AV,QUINT AVE,-122.39597,37.73780,3,NaN,NaN,NaN,NONE
2937,8575277,2018-02-21,14,Y,4,A,nan,C,B,A,...,NaN,16TH ST,FLORIDA ST,-122.41144,37.76555,1,NaN,NaN,NaN,NONE
394,8777287,2018-12-28,15,Y,4,A,nan,B,G,A,...,NaN,CONNECTICUT ST,CESAR CHAVEZ ST,-122.39627,37.74975,4,NaN,NaN,NaN,NONE
860,8754859,2018-10-24,8,N,3,A,nan,B,G,A,...,NaN,JUDAH ST,19TH AV,-122.47703,37.76170,4,NaN,NaN,NaN,NONE


# Load in Uber Speeds dataset

In [8]:
avgspeeds = pd.read_csv('avgspeeds.csv')
avgspeeds.head()

/Users/zacharybarnes/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,quarter,hour_of_day,osm_way_id,osm_start_node_id,osm_end_node_id,speed_mph_mean,speed_mph_stddev,speed_mph_p50,speed_mph_p85
0,2018,1,6,5.17293e+08,5.04823e+09,5.04823e+09,36.035,9.068,34.582,44.082
1,2018,1,9,5.17293e+08,5.04823e+09,5.04823e+09,23.818,13.096,22.628,36.568
2,2018,1,12,5.17293e+08,5.04823e+09,5.04823e+09,21.129,13.039,19.453,33.742
3,2018,1,12,5.17293e+08,5.04823e+09,5.04823e+09,30.38,9.873,29.324,40.707
4,2018,1,18,5.17293e+08,5.04823e+09,5.04823e+09,22.927,12.812,22.033,32.872


In [13]:
# Reduce cv DataFrame to only entries with non-null Quarter, Collision_Time, and OSM_ID 
# before joining to avgspeeds
cvsmall = cv.loc[(~cv['QUARTER'].isna()) & (~cv['COLLISION_TIME'].isna()) & ~(cv['OSM_ID'] == 'NONE')]
cvsmall['OSM_ID'] = cvsmall['OSM_ID'].astype(int)

# There are 271 entries.

/Users/zacharybarnes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/zacharybarnes/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,CASE_ID,COLLISION_DATE,COLLISION_TIME,INTERSECTION,COLLISION_SEVERITY,LIGHTING,LOCATION_TYPE,MVIW,TYPE_OF_COLLISION,WEATHER_1,...,LONGITUDE,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,QUARTER,VICTIM_DEGREE_OF_INJURY,VICTIM_ROLE,VICTIM_AGE,OSM_ID
251,8804185,2018-12-29,18,Y,2,C,nan,C,A,A,...,NaN,PORTOLA DR,CLIPPER ST,-122.444840,37.746940,4,2.0,1.0,32.0,148313526
254,8796720,2018-12-10,13,N,2,A,nan,C,D,A,...,NaN,SANSOME ST,CHESTNUT ST,-122.403498,37.805128,4,2.0,1.0,30.0,2503369217
260,8796604,2018-12-28,18,N,2,C,nan,C,D,A,...,NaN,BALBOA ST,4TH AV,-122.462364,37.777302,4,2.0,1.0,39.0,3802986672
261,8796085,2018-12-11,15,N,2,A,nan,C,D,A,...,NaN,BRYANT ST,4TH ST,-122.397722,37.779838,4,2.0,1.0,45.0,467150002
266,8787863,2018-12-12,9,N,2,A,nan,J,E,A,...,NaN,HICKORY ST,BUCHANAN ST,-122.428359,37.774759,4,2.0,2.0,998.0,27553863
272,8780730,2018-11-09,13,Y,2,A,nan,C,D,A,...,NaN,MANSELL ST,GOETTINGEN ST,-122.404060,37.720860,4,2.0,2.0,55.0,27652378
294,8777422,2018-12-10,18,N,2,D,nan,B,G,A,...,NaN,16TH ST,CONNECTICUT ST,-122.397802,37.766382,4,2.0,3.0,33.0,5595236177
295,8777421,2018-12-20,21,Y,2,C,nan,B,A,A,...,NaN,EDDY ST,LARKIN ST,-122.417520,37.783300,4,2.0,3.0,54.0,4625185531
335,8777359,2018-12-07,15,Y,2,A,nan,B,G,A,...,NaN,30TH AV,IRVING ST,-122.488900,37.763040,4,2.0,3.0,54.0,276880818
348,8777340,2018-12-01,22,N,2,C,nan,E,B,A,...,NaN,COLLEGE AV,MISSION ST,-122.424334,37.735083,4,2.0,3.0,998.0,27396883


The code below produces speeds.csv, a subset of averagespeeds.csv. It reduces the speeds observations down to ones that would match the Collisions-Victims dataset on OSD_ID, Quarter, and Collision_Time. If there is no match, set speed = 0.

In [ ]:
# speeds = list()
# for i, row in cvsmall.iterrows():
#     speed = avgspeeds.loc[(avgspeeds['osm_way_id'] == row['OSM_ID']) & 
#               (avgspeeds['quarter'] == row['QUARTER']) & 
#               (avgspeeds['hour_of_day'] == row['COLLISION_TIME'])]['speed_mph_mean']
#     if speed.values.size > 0:
#          speeds.append(speed.values[0])
        
#     else:
#         speeds.append(0)
# speeds = pd.Series(speeds)
# speeds.to_csv('speeds.csv')


Now, we join cvsmall (the reduced Collisions-Victims dataset) to the speeds dataset.

In [33]:
speeds = list(pd.read_csv("speeds.csv",header=None))
cvsmall["SPEEDS"] = speeds
print(len(speeds))

ValueError: Length of values does not match length of index

to do:
get rid of highway entries

In [25]:
cvsmall['VICTIM_DEGREE_OF_INJURY'].value_counts()

2.0    248
1.0     23
Name: VICTIM_DEGREE_OF_INJURY, dtype: int64

In [27]:
cvsmall['LOCATION_TYPE'].value_counts()

nan    261
H        5
I        4
R        1
Name: LOCATION_TYPE, dtype: int64